In [17]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm # Displays a progress bar
import random
import torch
import cv2
from torch import nn
from torch import optim
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import Dataset, Subset, DataLoader, random_split

In [18]:
torch.manual_seed(0)
random.seed(0)
np.random.seed(0)

In [19]:
print("Loading datasets...")
FASHION_transform = transforms.Compose([
      transforms.Resize((32,32)),
      transforms.ToTensor(),
])
FASHION_trainval = datasets.FashionMNIST('.', download=True, train=True, transform=FASHION_transform)
FASHION_train = Subset(FASHION_trainval, range(50000))
FASHION_val = Subset(FASHION_trainval, range(50000,60000))
FASHION_test = datasets.FashionMNIST('.', download=True, train=False, transform=FASHION_transform)
print("Done!")

Loading datasets...
Done!


In [76]:
trainloader = DataLoader(FASHION_train, batch_size=128, shuffle=True)
valloader = DataLoader(FASHION_val, batch_size=128, shuffle=True)
testloader = DataLoader(FASHION_test, batch_size=32, shuffle=True)

In [21]:
class Network(nn.Module):
    def __init__(self, num_classes):
        super(Network, self).__init__()
        # TODO: Design your own network, define layers here.
        # Here We provide a sample of two-layer fully-connected network from HW4 Part3.
        # Your solution, however, should contain convolutional layers.
        # Refer to PyTorch documentations of torch.nn to pick your layers. (https://pytorch.org/docs/stable/nn.html)
        # Some common Choices are: Linear, Conv2d, ReLU, MaxPool2d, AvgPool2d, Dropout
        # If you have many layers, consider using nn.Sequential() to simplify your code
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.fc = nn.Linear(400, 120)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(120, 84)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(84, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.relu(out)
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out

In [22]:
device = "cuda" if torch.cuda.is_available() else "cpu" # Configure device
print(device)
model = Network(10).to(device)
# TODO: Define loss function 
criterion = nn.CrossEntropyLoss() # Specify the loss layer
# TODO: Modify the line below, experiment with different optimizers and parameters (such as learning rate)
optimizer = optim.Adam(model.parameters(), lr=0.002, weight_decay=1e-4) # Specify optimizer and assign trainable parameters to it, weight_decay is L2 regularization strength
num_epoch = 26 # TODO: Choose an appropriate number of training epochs

cuda


In [126]:
def train(model, loader, num_epoch): 
    print("Start training...")
    model.train() 
    for i in range(num_epoch):
        running_loss = []
        for batch, label in tqdm(loader):
            batch = batch.to(device)
            label = label.to(device)
            optimizer.zero_grad() # Clear gradients from the previous iteration
            pred = model(batch) # This will call Network.forward() that you implement
            loss = criterion(pred, label) # Calculate the loss
            running_loss.append(loss.item())
            loss.backward() # Backprop gradients to all tensors in the network
            optimizer.step() # Update trainable weights
        print("Epoch {} loss:{}".format(i+1,np.mean(running_loss))) # Print the average loss for this epoch
    print("Done!")


In [127]:
train(model, trainloader, num_epoch)
print("Evaluate on validation set...")

Start training...


100%|█████████████████████████████████████████| 391/391 [00:05<00:00, 70.47it/s]


Epoch 1 loss:0.08204946103398605


100%|█████████████████████████████████████████| 391/391 [00:05<00:00, 70.61it/s]


Epoch 2 loss:0.08166411353746796


...

Epoch 24 loss:0.08187956695475847


100%|█████████████████████████████████████████| 391/391 [00:05<00:00, 68.94it/s]


Epoch 25 loss:0.0816751612955347


100%|█████████████████████████████████████████| 391/391 [00:05<00:00, 67.75it/s]

Epoch 26 loss:0.08164195743534723
Done!
Evaluate on validation set...


In [24]:
def fgsm(image, epsilon, data_grad):
    # Collect the element-wise sign of the data gradient
    sign_data_grad = data_grad.sign()
    # Create the perturbed image by adjusting each pixel of the input image
    perturbed_image = image + epsilon*sign_data_grad
    # Adding clipping to maintain [0,1] range
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    # Return the perturbed image
    return perturbed_image

In [25]:
def evaluate(model, loader, eps):
    model.eval() 
    correct = 0
    for batch, label in tqdm(loader):
        batch = batch.to(device)
        label = label.to(device)
        batch.requires_grad = True
        pred = model(batch)
        loss = criterion(pred, label)
        model.zero_grad()
        loss.backward()
        testgrad = batch.grad.data
        attackdata = fgsm(batch, eps, testgrad)
        pred = model(attackdata)
        correct += (torch.argmax(pred,dim=1)==label).sum().item()
    acc = correct/len(loader.dataset)
    print("Evaluation accuracy: {}".format(acc))
    return acc


In [110]:
def pgd_attack(model, image, labels, eps = 0.3, epsilon=25/255, iters=1) :
    image = image.to(device)
    labels = labels.to(device)
    loss = nn.CrossEntropyLoss()
    ori_image = image.data
    image = image + torch.empty_like(image).uniform_(-eps, eps)
    image = torch.clamp(image, min = 0, max = 1).detach()
    for i in range(iters) :    
        image.requires_grad = True
        output = model(image)
        model.zero_grad()
        cost = loss(output, labels).to(device)
        cost.backward()
        adv_image = image + epsilon * image.grad.sign()
        eta = torch.clamp(adv_image - ori_image, min = -eps, max = eps)
        image = torch.clamp(ori_image + eta, min = 0, max = 1).detach_()
    return image

In [130]:
model = torch.load("./saved_model.pth")

In [131]:
print("Evaluate on test set")
evaluate(model, testloader, 0)

Evaluate on test set


100%|████████████████████████████████████████| 313/313 [00:01<00:00, 182.57it/s]

Evaluation accuracy: 0.9033


0.9033

In [34]:
print("FGSM Attack")
test_acc = []
test_eps = [0,25/255]
# Run test for each epsilon
for eps in test_eps:
    acc= evaluate(model, testloader, eps)
    test_acc.append(acc)
    #examples.append(ex)

FGSM Attack


100%|████████████████████████████████████████| 313/313 [00:01<00:00, 187.48it/s]


Evaluation accuracy: 0.9027


100%|████████████████████████████████████████| 313/313 [00:01<00:00, 185.07it/s]

Evaluation accuracy: 0.1421


In [132]:
print("PGD Attack")
epsilon = 25/255
for iters in [1, 2, 5, 10]:
    correct = 0
    for image, target in testloader:
        image = pgd_attack(model, image, target, 0.3, epsilon, iters)
        target = target.to(device)
        output = model(image)
        pred = output.max(1, keepdim=False)[1]
        correct += (pred==target).sum().item()
        final_acc = correct/float(len(testloader))
    print("Epsilon: {}\tIterations: {}\tTest Attack Success Rate = {}%".format(epsilon, iters, (1 - final_acc) * 100))

PGD Attack
Epsilon: 0.09803921568627451	Iterations: 1	Test Attack Success Rate = 22.364217252396166%
Epsilon: 0.09803921568627451	Iterations: 2	Test Attack Success Rate = 97.76357827476039%
Epsilon: 0.09803921568627451	Iterations: 5	Test Attack Success Rate = 100.0%
Epsilon: 0.09803921568627451	Iterations: 10	Test Attack Success Rate = 100.0%


In [129]:
torch.save(model, "./saved_model.pth")